In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from xgboost.sklearn import XGBClassifier

In [ ]:
# Import the training dataset, supplied by Kaggle <https://www.kaggle.com/c/santander-product-recommendation/data>

train_ver2 = pd.read_csv("../../capstone_assets/train_ver2.csv")

In [ ]:
# Drop the columns never used in the study to reduce memory footprint. Identified by earlier EDA.

for col in ['conyuemp','tipodom','indrel','ult_fec_cli_1t','indfall']:
    train_ver2.drop(col,inplace=True,axis=1)

# Data cleaning, filling missing values to reduce unnecessary blank or false dummy variables.    
    
train_ver2['ind_empleado'].fillna('N',inplace=True)
train_ver2['pais_residencia'].fillna('ES',inplace=True)
train_ver2['tiprel_1mes'].fillna('I',inplace=True)
train_ver2['nomprov'].fillna('DESCONOCIDO',inplace=True)
train_ver2['segmento'].fillna('02 - PARTICULARES',inplace=True)

train_ver2['ind_empleado'].replace('S','P',inplace=True)
train_ver2['age'].replace(to_replace=' NA',value=np.NaN,inplace=True)
train_ver2.fecha_alta = train_ver2.fecha_alta.replace(to_replace=-9223372036854775808,value=np.NaN)
train_ver2['antiguedad'].replace(to_replace=[' NA','     NA'],value=np.NaN,inplace=True)
train_ver2.loc[train_ver2['antiguedad'] <0 , 'antiguedad'] = 0

for col in ['fecha_dato','fecha_alta']:
    train_ver2[col] = pd.to_datetime(train_ver2[col],yearfirst=True)

# Convert columns from Objects, to move optimised data types. Reduces memory footprint, particularly important 
# given the scale of the data
    
for col in ['ncodpers', 'age', 'fecha_alta', 'ind_nuevo', 'antiguedad','indrel_1mes']:
    train_ver2[col] = pd.to_numeric(train_ver2[col],downcast='unsigned',errors='ignore')
for col in ['ind_empleado', 'pais_residencia', 'sexo', 'tiprel_1mes', 'indresi', 'indext', 'canal_entrada','nomprov','ind_actividad_cliente','segmento']:
    train_ver2[col] = train_ver2[col].astype('category')

In [ ]:
# Loop through the product columns;
# * filling blanks (missing products are products customers do not have)
# * optimising the astype
# * and capturing their names (for operations later)

product_columns = []
for column in train_ver2.columns:
    if 'ind_' in column and '_ult1' in column:
        train_ver2[column].fillna(0,inplace=True)
        train_ver2[column] = train_ver2[column].astype('category')
        product_columns.append(column)

In [ ]:
# Defining the columns used in modelling, per category.
# Broken out so that they can be swapped in and out, and re-ran

id_columns = ['ncodpers']
date_columns = ['fecha_dato']
numerical_columns = ['fecha_alta','antiguedad','renta',]
categorical_columns = ['ind_empleado','sexo','indrel_1mes','indresi','segmento','ind_nuevo','ind_actividad_cliente','nomprov']

# These categorical columns are not included in the model, as the weight/dimensionality they add
# isn't worth the modelling value after initial experimentation. But, others may wish to add them back

reduced_categorical_columns = ['canal_entrada','pais_residencia','tiprel_1mes','indext']

In [ ]:
# The modelling is based off three dataframes
# *w, meaning wide, which is the dataframe of just the selected features, with categories dummied
train_ver2w = pd.concat([train_ver2[id_columns],train_ver2[date_columns],train_ver2[numerical_columns],pd.get_dummies(train_ver2[categorical_columns],dummy_na=True),train_ver2[product_columns]],axis=1)

# *b, meaning before, which is the training data
train_ver2b = train_ver2w[train_ver2w['fecha_dato'].isin(['28-04-2016'])]
train_ver2b.drop(['fecha_dato'], axis=1, inplace = True)

# *a, meaning after, which is the test data
train_ver2a = train_ver2w[train_ver2w['fecha_dato'].isin(['28-05-2016'])]
train_ver2a.drop(['fecha_dato'], axis=1, inplace = True)

In [ ]:
# Import the test data from kaggle.
# The test data customers in June, aligns with the *a data from the training data in May - exactly the same customers, IDs etc

test_ver2 = pd.read_csv("../../capstone_assets/test_ver2.csv")

In [ ]:
# Now the data expands.
# For each data set, (a + b), for each prior month, the product set of the customer, is appended.
# So each dataset has a full product history per customer
# This product history, although heavy in data terms, has the predictive value of boosting the MAP@7 score
# from ~2.3 to ~2.6, vs just using one month to another to predict.

dates = train_ver2.fecha_dato.value_counts().index.values

for date in dates[2:]:
    dftr_prev = train_ver2[train_ver2['fecha_dato'].isin([date])][product_columns + ['ncodpers']]
    train_ver2b = pd.merge(train_ver2b,dftr_prev, how='left', on=['ncodpers'], suffixes=('', '_'+str(date)[0:10]))
    
# The 'after' set does not include the very first month of historical data, so the number of months, and dimensions
# match across the two data sets.

for date in dates[1:16]:
    dftr_prev = train_ver2[train_ver2['fecha_dato'].isin([date])][product_columns + ['ncodpers']]
    train_ver2a = pd.merge(train_ver2a,dftr_prev, how='left', on=['ncodpers'], suffixes=('', '_'+str(date)[0:10]))    

In [ ]:
# define the training target, which is the may product portfolio per customer

y_train = pd.merge(train_ver2b[['ncodpers']],train_ver2[train_ver2['fecha_dato'].isin(['25-05-2016'])][['ind_ahor_fin_ult1'] + ['ncodpers']], how='left', on=['ncodpers'])
y_train.drop(['ncodpers'], axis=1, inplace = True)

In [ ]:
# to simplify operations, I constrain the training data set to the customers who exist in the test data.
# this drops thousands of customers, and potentially reduces predictive accuracy, but greatly increases calculation simplicity

two_key_months = pd.Series(train_ver2[train_ver2['fecha_dato'].isin(['28-05-2016','28-04-2016'])]['ncodpers'].value_counts())
two_key_months = two_key_months[two_key_months == 2]
train_ver2b = train_ver2b[train_ver2b['ncodpers'].isin(two_key_months.index.values)]

In [ ]:
# With the data cleaned, and training and test data defined, now the modelling.
# The objective is relatively simple, but computationally intensive.
# * We build a productive model for each product
# * For the training, the target is each product, per customer, for May 2016. The predictor variables are the 
#   April demographics, and the monthly product history of the customer back 15 months.
# * For the test, the target is each product, per customer, but June 2016. The predictors are the May demographics,
#   and the product history also going back 15 months.

# Then, using the model per product, per customer we rule out the products they already have, and predict
# the products for June with the highest proba, and export it using Kaggle's required format, for a top 1000 score

model_preds = {}
id_preds = defaultdict(list)

model_preds_train = {}
id_preds_train = defaultdict(list)

ids = train_ver2b['ncodpers'].values
testids = train_ver2a['ncodpers'].values

counter = 0

for c in product_columns:
    print(c)
    print datetime.now()    
    print(counter)
    counter += 1

    y_train = pd.merge(train_ver2b[['ncodpers']],train_ver2[train_ver2['fecha_dato'].isin(['28-05-2016'])][[c] + ['ncodpers']], how='left', on=['ncodpers'])
    y_train.drop(['ncodpers'],1,inplace=True)
    x_train = train_ver2b.drop(['ncodpers'],1,inplace=False)
    x_test = train_ver2a.drop(['ncodpers'],1,inplace=False)

    
    xgbcla = XGBClassifier(objective="binary:logistic", n_estimators=200,max_depth=4,min_child_weight=3, learning_rate=0.05)  
    xgbcla.fit(x_train.as_matrix(), y_train.values)  
        
    p_test = xgbcla.predict_proba(x_test.as_matrix())[:,1]
        
    model_preds[c] = p_test
    for id, p in zip(testids, p_test):
        id_preds[id].append(p)
    
already_active = {}

for row in train_ver2a[['ncodpers']+product_columns].values:
    row = list(row)
    id = row.pop(0)
    active = [c[0] for c in zip(product_columns, row) if c[1] > 0]
    already_active[id] = active

test_preds = {}
for id, p in id_preds.items():
    preds = [i[0] for i in sorted([i for i in zip(product_columns, p) if i[0] not in already_active[id]], key=lambda i:i [1], reverse=True)[:7]]
    test_preds[id] = preds

sample = pd.read_csv('../../capstone_assets/sample_submission.csv')

test_preds_prods = []
for row in sample.values:
    id = row[0]
    p = test_preds[id]
    test_preds_prods.append(' '.join(p))

sample['added_products'] = test_preds_prods
sample.to_csv('2017-06-19_submission.csv', index=False)